# ID2214/FID3214 Assignment 1 Group no. ID2214 - 5
### Project members:
Lukas Olenborg

Charlotte Jacquet

Isabella Rositi

## Load NumPy and pandas

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Filepath to your own drive folder and files if below doesnt work
filepath = '/content/drive/MyDrive/Colab Notebooks/ID2214/'

# Filepath to shared drive (you have to add the 'Programming for Data Science' -"shortcut" to your drive)
#filepath = '/content/drive/MyDrive/Programming for Data Science/Assignment 1/'

# FIlepath for TA, uncomment when submitting
#filepath = ''

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
from platform import python_version

print(f"Python version: {python_version()}")
print(f"NumPy version: {np.__version__}")
print(f"Pandas version: {pd.__version__}")

Python version: 3.8.16
NumPy version: 1.21.6
Pandas version: 1.3.5


## 1a. Create and apply column filter

In [ ]:
#@title
# Insert the functions create_column_filter and apply_column_filter below (after the comments)


# Input to create_column_filter:
# df - a dataframe (where the column names "CLASS" and "ID" have special meaning)
#
# Output from create_filter:
# df            - a new dataframe, where columns, except "CLASS" and "ID", containing only missing values
#                 or only one unique value (apart from the missing values) have been dropped
# column_filter - a list of the names of the remaining columns, including "CLASS" and "ID"
#
# Hint 1: First copy the input dataframe and modify the copy (the input dataframe should be kept unchanged)
#
# Hint 2: Iterate through all columns and consider to drop a column only if it is not labeled "CLASS" or "ID"
#
# Hint 3: You may check the number of unique (non-missing) values in a column by applying the pandas functions
#         dropna and unique to drop missing values and get the unique (remaining) values

def create_column_filter(df):

    # copy the input dataframe into df_copy
    df_copy = df.copy()

    for colname, values in df.iteritems():
        # only do something if it's not labeled 'CLASS' or 'ID'
        if colname == "CLASS" or colname == "ID":
          continue
        else:
            # Drop a column if it contains only missing values
            df_copy = df_copy.dropna(how = 'all', axis=1)

            # Drop a column if there's only one unique value...
            arr = df[colname].unique()
            # ... apart from the missing values
            arr = arr[~ pd.isnull(arr)] # '~' = np not operator (!), isnull = True where nan
            if arr.size == 1:
                #if colname in df_copy.columns: # this line replaced with errors='ignore' below
                df_copy = df_copy.drop([colname], axis=1, errors='ignore')
            #alternative apart from the missing values
            # elif arr.size == 2:
            #   for i in range(arr.size):
            #       if np.isnan(arr[i]):
            #           df_copy = df_copy.drop([colname],axis=1, errors='ignore')


    column_filter = df_copy.columns.values.tolist()

    return df_copy, column_filter

# Input to apply_column_filter:
# df            - a dataframe
# column_filter - a list of the names of the columns to keep (see above)
#
# Output from apply_column_filter:
# df - a new dataframe, where each column that is not included in column_filter has been dropped
#
# Hint 1: First copy the input dataframe and modify the copy (the input dataframe should be kept unchanged)

def apply_column_filter(df, column_filter):
    return df.loc[:,column_filter] # : for all rows , column_filter cols

In [ ]:
# Test your code (leave this part unchanged)

df = pd.DataFrame({"CLASS":[1,0,1,0,1],"A":[1,2,np.nan,4,5],"B":[1,1,1,1,np.nan],"C":["h","h",np.nan,"i","h"],"D":[np.nan,np.nan,np.nan,np.nan,np.nan]})

filtered_df, column_filter = create_column_filter(df)

new_df = pd.DataFrame({"CLASS":[1,0,0],"A":[4,5,6],"B":[1,2,1],"C":[np.nan,np.nan,np.nan],"D":[np.nan,4,5]})

filtered_new_df = apply_column_filter(new_df,column_filter)

display("df",df)


display("filtered_df",filtered_df)
display("new_df",new_df)
display("filtered_new_df",filtered_new_df)

'df'

,CLASS,A,B,C,D
0,1,1.0,1.0,h,NaN
1,0,2.0,1.0,h,NaN
2,1,NaN,1.0,NaN,NaN
3,0,4.0,1.0,i,NaN
4,1,5.0,NaN,h,NaN


'filtered_df'

,CLASS,A,C
0,1,1.0,h
1,0,2.0,h
2,1,NaN,NaN
3,0,4.0,i
4,1,5.0,h


'new_df'

,CLASS,A,B,C,D
0,1,4,1,NaN,NaN
1,0,5,2,NaN,4.0
2,0,6,1,NaN,5.0


'filtered_new_df'

,CLASS,A,C
0,1,4,NaN
1,0,5,NaN
2,0,6,NaN


## 1b. Create and apply normalization

In [ ]:
# Insert the functions create_normalization and apply_normalization below (after the comments)
#
# Input to create_normalization:
# df: a dataframe (where the column names "CLASS" and "ID" have special meaning)
# normalizationtype: "minmax" (default) or "zscore"
#
# Output from create_normalization:
# df            - a new dataframe, where each numeric value in a column has been replaced by a normalized value
# normalization - a mapping (dictionary) from each column name to a triple, consisting of
#                ("minmax",min_value,max_value) or ("zscore",mean,std)
#
# Hint 1: First copy the input dataframe and modify the copy (the input dataframe should be kept unchanged)
#
# Hint 2: Consider columns of type "float" or "int" only (and which are not labeled "CLASS" or "ID"),
#         the other columns should remain unchanged
#
# Hint 3: Take a close look at the lecture slides on data preparation

def create_normalization(df, normalizationtype):
    # copy the input dataframe into df_copy
    df_copy = df.copy()
    mapping = {}
    for colname, values in df_copy.iteritems():

        # only do something if it's not labeled 'CLASS' or 'ID'
        if colname == "CLASS" or colname == "ID":
          continue
        else :
            # Consider column of type "float" or "int"
            # Normalization of the value
            if df_copy[colname].dtype == 'float64' or 'int32' or 'int64' or 'float32':
                # dictionary from each column name to a triple ("minmax",min_value,max_value) or ("zscore",mean,std)
                if normalizationtype == "zscore":
                    # z normalization
                    mean = df_copy[colname].mean()
                    std = df_copy[colname].std()
                    #apply z normalization
                    df_copy[colname] = df_copy[colname].apply(lambda x: (x-mean)/std)
                    #filling the dictionary
                    triple = ["zscore", mean, std]
                    mapping[colname] = triple


                else :
                    #min-max normalization
                    min = df_copy[colname].min()
                    max = df_copy[colname].max()
                    # apply min-max normalization
                    df_copy[colname] = [(x-min)/(max-min) for x in df_copy[colname]]
                    # store max,mean and mean for each column
                    # for each column dictionary with three values (nested dictionary)
                    triple = ["minmax", min, max]
                    mapping[colname] = triple

    print(mapping)

    return df_copy, mapping


# Input to apply_normalization:
# df            - a dataframe
# normalization - a mapping (dictionary) from column names to triples (see above)
#
# Output from apply_normalization:
# df - a new dataframe, where each numerical value has been normalized according to the mapping
#
# Hint 1: First copy the input dataframe and modify the copy (the input dataframe should be kept unchanged)
#
# Hint 2: For minmax-normalization, you may consider to limit the output range to [0,1]

def apply_normalization(df, mapping):
    # copy the input dataframe into df_copy
    df_copy = df.copy()

    for colname, values in df_copy.iteritems():
        # only do something if it's not labeled 'CLASS' or 'ID'
        if colname == "CLASS" or colname == "ID":
          continue;
        else :
            # Consider column of type "float" or "int"
            # Normalization of the value
            if df_copy[colname].dtype == 'float64' or 'int32' or 'int64' or 'float32':

                if mapping[colname][0] == "zscore":
                    # z normalization
                    mean = mapping[colname][1]
                    std = mapping[colname][2]
                    #apply z normalization
                    df_copy[colname] = df_copy[colname].apply(lambda x: (x-mean)/std)

                else :
                    #min-max normalization
                    min = mapping[colname][1]
                    max = mapping[colname][2]
                    # apply min-max normalization
                    # limit the output range to [0,1]
                    df_copy[colname] = [1 if (x-min)/(max-min) > 1 else 0 if (x-min)/(max-min) < 0 else (x-min)/(max-min) for x in df_copy[colname]]


    return df_copy

In [ ]:
glass_train_df = pd.read_csv(filepath + "glass_train.csv")
glass_test_df = pd.read_csv(filepath + "glass_test.csv")

glass_train_norm, normalization = create_normalization(glass_train_df,normalizationtype="minmax")
print("normalization:\n")
print(glass_train_norm)
for f in normalization:
    print("{}:{}".format(f,normalization[f]))

print()

glass_test_norm = apply_normalization(glass_test_df,normalization)
display("glass_test_norm",glass_test_norm)



{'RI': ['minmax', 1.51131, 1.53125], 'Na': ['minmax', 10.73, 15.79], 'Mg': ['minmax', 0.0, 4.49], 'Al': ['minmax', 0.29, 3.04], 'Si': ['minmax', 69.81, 75.18], 'K': ['minmax', 0.0, 6.21], 'Ca': ['minmax', 5.43, 14.68], 'Ba': ['minmax', 0.0, 3.15], 'Fe': ['minmax', 0.0, 0.37]}
normalization:

      ID        RI        Na        Mg        Al        Si         K  \
0    202  0.261785  0.241107  0.000000  0.327273  1.000000  0.434783   
1    124  0.288867  0.543478  0.775056  0.516364  0.504655  0.099839   
2    152  0.499498  0.709486  0.868597  0.196364  0.314711  0.000000   
3    197  0.213139  0.620553  0.000000  0.818182  0.636872  0.022544   
4    144  0.289870  0.448617  0.772829  0.545455  0.541899  0.106280   
..   ...       ...       ...       ...       ...       ...       ...   
102  178  0.404213  0.604743  0.536748  0.327273  0.549348  0.000000   
103  160  0.333501  0.547431  0.748330  0.487273  0.396648  0.091787   
104   88  0.257773  0.527668  0.777283  0.447273  0.528864 

'glass_test_norm'

,ID,RI,Na,Mg,Al,Si,K,Ca,Ba,Fe,CLASS
0,101,0.262788,0.399209,0.634744,0.418182,0.644320,0.091787,0.363243,0.034921,0.594595,2
1,104,0.799398,0.606719,0.701559,0.134545,0.141527,0.012882,0.671351,0.000000,0.000000,2
2,44,0.541123,0.592885,0.855234,0.156364,0.363128,0.027375,0.465946,0.000000,0.000000,1
3,17,0.327482,0.385375,0.817372,0.316364,0.614525,0.098229,0.353514,0.000000,0.000000,1
4,81,0.231194,0.420949,0.783964,0.665455,0.530726,0.111111,0.274595,0.000000,0.000000,2
...,...,...,...,...,...,...,...,...,...,...,...
102,80,0.230191,0.413043,0.783964,0.585455,0.567970,0.111111,0.274595,0.000000,0.000000,2
103,141,0.280341,0.513834,0.788419,0.480000,0.508380,0.109501,0.289730,0.000000,0.000000,2
104,14,0.309428,0.420949,0.792873,0.356364,0.633147,0.086957,0.318919,0.000000,0.459459,1
105,132,0.743731,0.586957,0.000000,0.389091,0.266294,0.030596,0.865946,0.000000,0.270270,2


## 1c. Create and apply imputation

In [ ]:
# Insert the functions create_imputation and apply_imputation below (after the comments)
#
# Input to create_imputation:
# df: a dataframe (where the column names "CLASS" and "ID" have special meaning)
#
# Output from create_imputation:
# df         - a new dataframe, where each missing numeric value in a column has been replaced by the mean of that column
#              and each missing categoric value in a column has been replaced by the mode of that column
# imputation - a mapping (dictionary) from column name to value that has replaced missing values
#
# Hint 1: First copy the input dataframe and modify the copy (the input dataframe should be kept unchanged)
#
# Hint 2: Handle columns of type "float" or "int" only (and which are not labeled "CLASS" or "ID") in one way
#         and columns of type "object" and "category" in other ways
#
# Hint 3: Consider using the pandas functions mean and mode respectively, as well as fillna
#
# Hint 4: In the rare case of all values in a column being missing*, replace numeric values with 0,
#         object values with "" and category values with the first category (cat.categories[0])
#
#         *Note that this will not occur if the previous column filter function has been applied
def create_imputation(df):
    dfcpy = df.copy()
    imputation = {}

    for col in df.columns:
        if col == "ID" or col == "CLASS":
            continue

        coltype = dfcpy[col].dtypes # get type of column

        if coltype == "int64" or coltype == "float64":
            # Replace nan values with mean of column for numeric cols
            mean = np.mean(dfcpy[col])

            # Rare case - all values nan
            if np.isnan(mean):
                mean = 0

            dfcpy[col] = dfcpy[col].fillna(mean)
            imputation[col] = mean

        if coltype == "object" or coltype == "category":
            # Replace nan values with mode (most common value) for object/catg columns
            mode = dfcpy[col].mode()

            # Rare case - all values nan (the category one in Hint 4 doesnt make sense
            # since category[0] = nan ...)
            if mode.empty:
                mode = ""
            else:
                mode = mode[0]

            dfcpy[col] = dfcpy[col].fillna(mode)
            imputation[col] = mode

    return dfcpy, imputation

# Input to apply_imputation:
# df         - a dataframe
# imputation - a mapping (dictionary) from column name to value that should replace missing values
#
# Output from apply_imputation:
# df - a new dataframe, where each missing value has been replaced according to the mapping
#
# Hint 1: First copy the input dataframe and modify the copy (the input dataframe should be kept unchanged)
#
# Hint 2: Consider using fillna

def apply_imputation(df, imputation):
    dfcpy = df.copy()
    for col in df.columns:
        if col == "CLASS" or col == "ID":
            continue

        # Check if this col was imputed
        if col in imputation:
            value = imputation[col] # get imputation (mean or mode)
            dfcpy[col] = dfcpy[col].fillna(value)

    return dfcpy

## Tests
#data = {
#    "col1":[5,np.nan,10,20],
#    "col2":pd.Series([np.nan,np.nan,np.nan,np.nan],dtype="category"),
#    "col3":pd.Series([np.nan,np.nan,np.nan,np.nan],dtype="object")
#    }
#dftest = pd.DataFrame(data)
#dftest[col] = dftest[col].fillna(dftest[col].cat.categories[0])
#print(dftest)
#test = create_imputation(dftest)
#print()
#print(test[0])

In [ ]:
# Test your code (leave this part unchanged)

anneal_train_df = pd.read_csv(filepath + "anneal_train.csv")
anneal_test_df = pd.read_csv(filepath + "anneal_test.csv")

anneal_train_imp, imputation = create_imputation(anneal_train_df)
anneal_test_imp = apply_imputation(anneal_test_df,imputation)

print("Imputation:\n")
for f in imputation:
    print("{}:{}".format(f,imputation[f]))

print("\nNo. of replaced missing values in training data:\n{}".format(anneal_train_imp.count()-anneal_train_df.count()))
print("\nNo. of replaced missing values in test data:\n{}".format(anneal_test_imp.count()-anneal_test_df.count()))

Imputation:

family:TN
product-type:C
steel:A
carbon:3.859688195991091
hardness:13.084632516703786
temper_rolling:T
condition:S
formability:2.2517482517482517
strength:26.302895322939868
non-ageing:N
surface-finish:P
surface-quality:E
enamelability:1.7142857142857142
bc:Y
bf:Y
bt:Y
bw/me:B
bl:Y
m:0
chrom:C
phos:P
cbond:Y
marvi:0
exptl:0
ferro:Y
corr:0
blue-bright-varn-clean:B
lustre:Y
jurofm:0
s:0
p:0
shape:SHEET
thick:1.1911937639198218
width:769.4917594654788
len:1229.293986636971
oil:Y
bore:35.18930957683742
packing:3.0

No. of replaced missing values in training data:
family                    382
product-type                0
steel                      43
carbon                      0
hardness                    0
temper_rolling            374
condition                 160
formability               163
strength                    0
non-ageing                391
surface-finish            444
surface-quality           128
enamelability             442
bc                        448
b

### Comment on assumptions, things that do not work properly, etc.

The rare case in Hint 4 for categories is not implemented. cat.category[0] is the first category of a column, and in this case all values would be nan, so this first category is not defined. Instead, empty strings are imputed, similar to objects case.

## 1d. Create and apply discretization

In [ ]:
# Insert the functions create_bins and apply_bins below
#
# Input to create_bins:
# df      - a dataframe
# nobins  - no. of bins (default = 10)
# bintype - either "equal-width" (default) or "equal-size"
#
# Output from create_bins:
# df      - a new dataframe, where each numeric feature value has been replaced by a categoric (corresponding to some bin)
# binning - a mapping (dictionary) from column name to bins (threshold values for the bin)
#
# Hint 1: First copy the input dataframe and modify the copy (the input dataframe should be kept unchanged)
#
# Hint 2: Discretize columns of type "float" or "int" only (and which are not labeled "CLASS" or "ID")
#
# Hint 3: Consider using pd.cut and pd.qcut respectively, with labels=False and retbins=True
#
# Hint 4: Set all columns in the new dataframe to be of type "category"
#
# Hint 5: Set the categories of the discretized features to be [0,...,nobins-1]
#
# Hint 6: Change the first and the last element of each binning to -np.inf and np.inf respectively
#
# Input to apply_bins:
# df      - a dataframe
# binning - a mapping (dictionary) from column name to bins (threshold values for the bin)


def create_bins(df, nobins = 10, bintype="equal-width"):
    # copy the input dataframe into df_copy and create an empty dictionary
    df_copy = df.copy()
    binning = {}

    # only do something if the column is not labeled 'CLASS' or 'ID'
    for colname, values in df_copy.iteritems():
        if colname == "CLASS" or colname == "ID":
            continue;
        else :
            # only consider column of type "float" or "int"
            if df_copy[colname].dtype == 'float64' or 'int32' or 'int64' or 'float32':
                # create the categories for each value and the tresholds for the bins
                if bintype == "equal-width":
                    cat = pd.cut(df_copy[colname], bins=nobins, labels=False, retbins = True, duplicates = "drop")
                elif bintype == "equal-size":
                    cat = pd.qcut(df_copy[colname], q=nobins, labels=False, retbins = True, duplicates = "drop")
                # replace with the categories and change column type to "category"
                df_copy[colname] = cat[0].astype('category')
                # change the first and the last element of each binning to -np.inf and np.inf
                cat[1][0] = -np.inf
                cat[1][-1] = np.inf
                # add to dictionary
                binning[colname] = cat[1]
    return df_copy, binning



# Output from apply_bins:
# df - a new dataframe, where each numeric feature value has been replaced by a categoric (corresponding to some bin)
#
# Hint 1: First copy the input dataframe and modify the copy (the input dataframe should be kept unchanged)
#
# Hint 2: Consider using pd.cut
#
# Hint 3: Set all columns in the new dataframe to be of type "category"
#
# Hint 4: Set the categories of the discretized features to be [0,...,nobins-1]

def apply_bins(df, binning):
    # copy the input dataframe into df_copy
    df_copy = df.copy()

    for colname, values in df_copy.iteritems():
       # only do something if the column is already in the dictionary
       for key, value in list(binning.items()):
            if colname == key:
                # create the category based on the bins provided in binning
                cat = pd.cut(df_copy[colname], bins=binning[colname], labels=False, retbins = False, duplicates = "drop")
                # replace with the categories and change type to "category"
                df_copy[colname] = cat.astype('category')
    return df_copy

In [ ]:
# Test your code  (leave this part unchanged)

glass_train_df = pd.read_csv(filepath + "glass_train.csv")
glass_test_df = pd.read_csv(filepath + "glass_test.csv")

glass_train_disc, binning = create_bins(glass_train_df, nobins=10, bintype="equal-size")
print("binning:")
for f in binning:
    print("{}:{}".format(f,binning[f]))

print()
glass_test_disc = apply_bins(glass_test_df, binning)
display("glass_test_disc", glass_test_disc)

binning:
RI:[    -inf 1.515896 1.51618  1.516516 1.516866 1.51753  1.517902 1.518618
 1.520114 1.521846      inf]
Na:[  -inf 12.73  12.872 13.    13.222 13.38  13.492 13.794 14.198 14.82
    inf]
Mg:[ -inf 1.82  3.188 3.41  3.476 3.55  3.61  3.728   inf]
Al:[ -inf 0.906 1.172 1.23  1.348 1.48  1.54  1.622 1.808 2.094   inf]
Si:[  -inf 71.756 72.196 72.388 72.72  72.79  72.966 73.06  73.208 73.372
    inf]
K:[ -inf 0.006 0.148 0.39  0.54  0.576 0.6   0.636 0.67    inf]
Ca:[  -inf  7.978  8.112  8.338  8.554  8.67   8.81   9.032  9.674 10.924
    inf]
Ba:[-inf 0.78  inf]
Fe:[ -inf 0.062 0.118 0.24    inf]



'glass_test_disc'

,ID,RI,Na,Mg,Al,Si,K,Ca,Ba,Fe,CLASS
0,101,3,1,1,4,8,4,5,0,2,2
1,104,9,7,1,0,0,1,9,0,0,2
2,44,9,6,7,0,1,2,8,0,0,1
3,17,5,0,6,1,7,6,5,0,0,1
4,81,1,1,4,9,3,8,0,0,0,2
...,...,...,...,...,...,...,...,...,...,...,...
102,80,1,1,4,8,5,8,0,0,0,2
103,141,4,4,4,6,3,8,1,0,0,2
104,14,4,1,5,3,8,3,3,0,2,1
105,132,9,6,0,4,0,2,9,0,1,2


In [ ]:
#Check that for every column the first category is equal to 0 and the last is equal to bin-1

for colname, values in glass_test_disc.iteritems():
    if colname != "ID" and colname != "CLASS":
        print(colname)
        print(values.unique(), "\n")

RI
[3, 9, 5, 1, 6, 4, 0, 8, 2, 7]
Categories (10, int64): [0, 1, 2, 3, ..., 6, 7, 8, 9] 

Na
[1, 7, 6, 0, 3, 4, 5, 9, 8, 2]
Categories (10, int64): [0, 1, 2, 3, ..., 6, 7, 8, 9] 

Mg
[1, 7, 6, 4, 5, 0, 3, 2]
Categories (8, int64): [0, 1, 2, 3, 4, 5, 6, 7] 

Al
[4, 0, 1, 9, 2, 8, 3, 5, 7, 6]
Categories (10, int64): [0, 1, 2, 3, ..., 6, 7, 8, 9] 

Si
[8, 0, 1, 7, 3, 5, 9, 6, 2, 4]
Categories (10, int64): [0, 1, 2, 3, ..., 6, 7, 8, 9] 

K
[4, 1, 2, 6, 8, 7, 5, 0, 3]
Categories (9, int64): [0, 1, 2, 3, ..., 5, 6, 7, 8] 

Ca
[5, 9, 8, 0, 3, 1, 2, 7, 4, 6]
Categories (10, int64): [0, 1, 2, 3, ..., 6, 7, 8, 9] 

Ba
[0, 1]
Categories (2, int64): [0, 1] 

Fe
[2, 0, 1, 3]
Categories (4, int64): [0, 1, 2, 3] 



## 1e. Create and apply one-hot encoding

In [ ]:
# Insert the functions create_one_hot and apply_one_hot below
#
# Input to create_one_hot:
# df: a dataframe
#
# Output from create_one_hot:
# df      - a new dataframe, where each categoric feature has been replaced by a set of binary features
#           (as many new features as there are possible values)
# one_hot - a mapping (dictionary) from column name to a set of categories (possible values for the feature)
#
# Hint 1: First copy the input dataframe and modify the copy (the input dataframe should be kept unchanged)
#
# Hint 2: Consider columns of type "object" or "category" only (and which are not labeled "CLASS" or "ID")
#
# Hint 3: Consider creating new column names by merging the original column name and the categorical value
#
# Hint 4: Set all new columns to be of type "float"
#
# Hint 5: Do not forget to remove the original categoric feature


def create_one_hot(df):
    # copy the input dataframe into df_copy and create an empy dictionary
    df_copy = df.copy()
    one_hot = {}
    # only do something if the column is not labeled 'CLASS' or 'ID'
    for colname, values in df_copy.iteritems():
        if colname == "CLASS" or colname == "ID":
            continue;
        else :
            # consider column of type "object" or "category"
            if df_copy[colname].dtype == 'object' or 'category':
                # get the values and put them in the dictionary
                one_hot[colname] = df_copy[colname].unique()
                # create new columns with the one_hot encoding
                for val in values:
                    df_copy[colname+"_"+val] = df_copy[colname].apply(lambda x: 1 if x==val else 0)
                # drop the old column
                df_copy.drop(colname, axis=1, inplace=True)
    return df_copy, one_hot


# Input to apply_one_hot:
# df      - a dataframe
# one_hot - a mapping (dictionary) from column name to categories
#
# Output from apply_one_hot:
# df - a new dataframe, where each categoric feature has been replaced by a set of binary features
#
# Hint: See the above Hints

def apply_one_hot(df, one_hot):
    # copy the input dataframe into df_copy
    df_copy = df.copy()
    for colname, values in df_copy.iteritems():
        for key, val in list(one_hot.items()):
            # only do something if the column is already in the dictionary
            if colname == key:
                #create new columns with the one_hot encoding
                for i, v in enumerate(val):
                    df_copy[key+"_"+v] = df_copy[key].apply(lambda x: 1 if x==v else 0)
                #drop the old column
                df_copy.drop(key, axis=1, inplace=True)
    return df_copy

In [ ]:
# Test your code  (leave this part unchanged)

train_df = pd.read_csv(filepath+"tic-tac-toe_train.csv")

new_train, one_hot = create_one_hot(train_df)

test_df = pd.read_csv(filepath+"tic-tac-toe_test.csv")

new_test_df = apply_one_hot(test_df, one_hot)
display("new_test_df", new_test_df)

'new_test_df'

,CLASS,top-left-square_o,top-left-square_b,top-left-square_x,top-middle-square_o,top-middle-square_b,top-middle-square_x,top-right-square_o,top-right-square_b,top-right-square_x,...,middle-right-square_x,bottom-left-square_o,bottom-left-square_x,bottom-left-square_b,bottom-middle-square_x,bottom-middle-square_b,bottom-middle-square_o,bottom-right-square_x,bottom-right-square_o,bottom-right-square_b
0,positive,1,0,0,1,0,0,0,0,1,...,1,1,0,0,1,0,0,1,0,0
1,positive,1,0,0,1,0,0,0,0,1,...,1,0,1,0,0,0,1,1,0,0
2,positive,0,1,0,0,0,1,0,0,1,...,0,0,1,0,0,1,0,0,1,0
3,negative,1,0,0,0,0,1,0,0,1,...,0,0,1,0,1,0,0,0,1,0
4,positive,0,0,1,1,0,0,1,0,0,...,0,0,0,1,0,0,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
474,positive,0,1,0,0,1,0,0,0,1,...,1,1,0,0,0,1,0,1,0,0
475,negative,1,0,0,1,0,0,1,0,0,...,1,0,0,1,1,0,0,0,0,1
476,positive,0,0,1,0,1,0,0,0,1,...,0,0,1,0,0,0,1,0,1,0
477,positive,0,0,1,0,1,0,0,0,1,...,0,0,1,0,0,0,1,0,1,0


## 1f. Divide a dataset into a training and a test set

In [ ]:
# Insert the function split below
#
# Input to split:
# df           - a dataframe
# testfraction - a float in the range (0,1) (default = 0.5)
#
# Output from split:
# trainingdf - a dataframe consisting of a random sample of (1-testfraction) of the rows in df
# testdf     - a dataframe consisting of the rows in df that are not included in trainingdf
#
# Hint: You may use np.random.permutation(df.index) to get a permuted list of indexes where a
#       prefix corresponds to the test instances, and the suffix to the training instances

def split(df,testfraction):
    # number of samples
    row = df.shape[0]
    trainingfraction = row*(1-testfraction)

    # a random permuted list of indexes
    # return an array containing random indexes
    random_list =  np.random.permutation(df.index)

    # initialize training and test dataframe
    trainingdf = pd.DataFrame([])
    testdf = pd.DataFrame([])
    # iterate over the samples (over the rows)
    for index, row in df.iterrows() :
        # put in training df the fisrt random (1-testfraction) of the rows
        if index < round(trainingfraction):
            trainingdf = trainingdf.append(df.iloc[random_list[index]], ignore_index = False)
        # put in test the rest of the samples
        else:
            testdf = testdf.append(df.iloc[random_list[index]], ignore_index = False)

    return trainingdf, testdf

In [ ]:
# Test your code  (leave this part unchanged)

glass_df = pd.read_csv(filepath + "glass.csv")

glass_train, glass_test = split(glass_df,testfraction=0.25)

print("Training IDs:\n{}".format(glass_train["ID"].values))

print("\nTest IDs:\n{}".format(glass_test["ID"].values))

print("\nOverlap: {}".format(set(glass_train["ID"]).intersection(set(glass_test["ID"]))))

Training IDs:
[189. 105. 152.  94. 133. 205. 170.  63. 141. 153.  85. 144.   4.  47.
  69.   3.  66. 163.  38. 198.  30.  99.  28.  36. 143.  83. 162.  17.
  74. 156.  11.  45. 158. 174. 101.  26.   9.  13.  72.  37.  76.  58.
 173.  32. 201. 190.  86.  39. 157.  20. 199. 196.  21.  48. 115.   2.
 208. 126.  42.  61.  25.  75.  54.  57. 202.  56. 111. 121.  65. 160.
  82. 182. 120. 124. 127. 164. 131.  81.  15. 195.  98.  55.  27. 146.
  23.  73. 210.  90. 135.  12.  24.  50.  53.  31. 172. 194.  79. 110.
 168.  34. 212. 175. 130. 203. 128.  77. 179. 140.  89.  62. 142. 155.
   6. 204. 148.  51.  84. 145. 171. 176. 138. 116. 211.  92. 132.   5.
 149.  49. 209. 106.  70. 125. 192.  16. 129. 104.  80. 159. 119. 147.
  78. 200. 139. 183. 214.  91.  97.  67. 187.   1. 213. 185.  29.  60.
 166. 113. 188. 109. 206.  33.]

Test IDs:
[167. 107. 178.  44. 114. 100.  87.  35. 118.  52. 169. 193. 165.  59.
 103.  64.  93. 134. 180. 197. 181. 186. 191. 102. 161.  41.  19.  22.
 136.   7. 122. 177.

## 1g. Calculate accuracy of a set of predictions

In [ ]:
# Insert the function accuracy below
#
# Input to accuracy:
# df            - a dataframe with class labels as column names and each row corresponding to
#                 a prediction with estimated probabilities for each class
# correctlabels - an array (or list) of the correct class label for each prediction
#                 (the number of correct labels must equal the number of rows in df)
#
# Output from accuracy:
# accuracy - the fraction of cases for which the predicted class label coincides with the correct label
#
# Hint: In case the label receiving the highest probability is not unique, you may
#       resolve that by picking the first (as ordered by the column names) or
#       by randomly selecting one of the labels with highest probaility.

def accuracy(df,correctlabels):

    preds = df.idxmax(axis=1) # get index(col) of max value on axis=1(row-wise)

    correct = np.sum(preds == correctlabels) # get nr of equal values in two arrays

    acc = correct/len(correctlabels)

    return acc


In [ ]:
# Test your code  (leave this part unchanged)

predictions = pd.DataFrame({"A":[0.5,0.5,0.5,0.25,0.25],"B":[0.5,0.25,0.25,0.5,0.25],"C":[0.0,0.25,0.25,0.25,0.5]})
display("predictions",predictions)

'predictions'

,A,B,C
0,0.50,0.50,0.00
1,0.50,0.25,0.25
2,0.50,0.25,0.25
3,0.25,0.50,0.25
4,0.25,0.25,0.50


In [ ]:
correctlabels = ["B","A","B","B","C"]

print("Accuracy: {}".format(accuracy(predictions,correctlabels))) # Note that depending on how ties are resolved the accuracy may be 0.6 or 0.8

Accuracy: 0.6


## 2a. Divide a dataset into a number of folds

In [ ]:
# Insert the function folds below
#
# Input to folds:
# df      - a dataframe
# nofolds - an integer greater than 1 (default = 10)
#
# Output from folds:
# folds - a list (of length = nofolds) of dataframes consisting of random non-overlapping,
#         approximately equal-sized subsets of the rows in df
#
# Hint: You may use np.random.permutation(df.index) to get a permuted list of indexes from which a
#       prefix corresponds to the test instances, and the suffix to the training instances

def folds(df, nofolds):
    print(f"nofolds: {nofolds}")
    print(f"df len: {len(df)}")
    dfs = []

    # Permutate df row indices
    idx_list = np.random.permutation(df.index)

    # Calculate len of fold
    fold_len = round(len(df)/nofolds)
    for i in range(nofolds-1):

        # Get folds slice indices [start,end]
        start = i*fold_len
        end = (i+1)*fold_len

        # Get permutated indices for fold
        fold_idx = idx_list[start:end]

        dfs.append(df.loc[fold_idx])

    # Handle last one separately
    fold_idx = idx_list[(i+1)*fold_len:]
    dfs.append(df.loc[fold_idx])

    return dfs

In [ ]:
# Test your code  (leave this part unchanged)

glass_df = pd.read_csv(filepath + "glass.csv")

glass_folds = folds(glass_df, nofolds=5)

fold_sizes = [len(f) for f in glass_folds]

print("Fold sizes:{}\nTotal no. instances: {}".format(fold_sizes,sum(fold_sizes)))

nofolds: 5
df len: 214
Fold sizes:[43, 43, 43, 43, 42]
Total no. instances: 214


## 2b. Calculate Brier score of a set of predictions

In [ ]:
# Insert the function brier_score below
#
# Input to brier_score:
# df            - a dataframe with class labels as column names and each row corresponding to
#                 a prediction with estimated probabilities for each class
# correctlabels - an array (or list) of the correct class label for each prediction
#                 (the number of correct labels must equal the number of rows in df)
#
# Output from brier_score:
# brier_score - the average square error of the predicted probabilties
#
# Hint: Compare each predicted vector to a vector for each correct label, which is all zeros except
#       for at the index of the correct class. The index can be found using np.where(df.columns==l)[0]
#       where l is the correct label.

def brier_score(df, correctlabels):
    #create matrix of zeros for true classes
    t_class = np.zeros((len(df), len(np.unique(correctlabels))))

    for colname, values in df.iteritems():
        #get the index of the true class
        ind = np.where(df.columns==colname)[0]

        for i, v in enumerate(correctlabels):
            # find the true class and change the values from 0 to 1
            if v == colname:
                t_class[i][ind] = 1

    # brier_score
    brier = np.mean(np.sum((df - t_class)**2, axis=1))

    return brier

In [ ]:
# Test your code  (leave this part unchanged)

predictions = pd.DataFrame({"A":[0.5,0.5,0.5,0.25,0.25],"B":[0.5,0.25,0.25,0.5,0.25],"C":[0.0,0.25,0.25,0.25,0.5]})

correctlabels = ["B","A","B","B","C"]

print("Brier score: {}".format(brier_score(predictions,correctlabels)))

Brier score: 0.5


## 2c. Calculate AUC of a set of predictions

In [ ]:
# Insert the function auc below
#
# Input to auc:
# df            - a dataframe with class labels as column names and each row corresponding to
#                 a prediction with estimated probabilities for each class
# correctlabels - an array (or list) of the correct class label for each prediction
#                 (the number of correct labels must equal the number of rows in df)
#
# Output from auc:
# auc - the weighted area under ROC curve
#
# Hint 1: Calculate the binary AUC first for each class label c, i.e., treating the
#         predicted probability of this class for each instance as a score; the true positives
#         are the ones belonging to class c and the false positives the rest
#
# Hint 2: When calculating the binary AUC, first find the scores of the true positives and then
#         the scores of the true negatives
#
# Hint 3: You may use a dictionary with a mapping from each score to an array of two numbers;
#         the number of true positives with this score and the number of true negatives with this score
#
# Hint 4: Created a (reversely) sorted (on the scores) list of pairs from the dictionary and
#         iterate over this to additively calculate the AUC
#
# Hint 5: For each pair in the above list, there are three cases to consider; the no. of false positives
#         is zero, the no. of true positives is zero, and both are non-zero
#
# Hint 6: Calculate the weighted AUC by summing the individual AUCs weighted by the relative
#         frequency of each class (as estimated from the correct labels)
def auc(df,correctlabels):
    all_dicts = []

    # Create triple dictionaries for each class
    for col in df.columns:

        # Initialize dictionary with s_i --> [0,0]
        triples = {}
        for prob in df[col]:
            triples[prob] = [0,0]

        # Generate dictionary
        i = 0
        for prob in df[col]:
            # number of istances in that class
            if correctlabels[i] == col:
                triples[prob][0] += 1
            # number of istances not in that class
            else:
                triples[prob][1] += 1
            i+=1

        print(f"Triples for {col}:")
        print(triples)
        print()
        all_dicts.append(triples)

    AUCs = [] # list for all AUCs

    # Calculate AUC for each triple dictionary
    for dicts in all_dicts:

        # Calculate total tps and fps
        tot_tp = 0
        tot_fp = 0
        print("Probability ordering:")
        for prob in sorted(dicts, reverse=True):
            print(prob)
            pair = dicts[prob]
            tot_tp += pair[0]
            tot_fp += pair[1]
        print()

        # Begin AUC algorithm
        single_AUC = 0
        cov_tp = 0

        # Loop through the sorted pairs
        for prob in sorted(dicts, reverse=True):
            pair = dicts[prob]
            tp = pair[0]
            fp = pair[1]

            if fp == 0:
                cov_tp += tp
            elif tp == 0:
                single_AUC += (cov_tp/tot_tp) * (fp/tot_fp)
            else:
                single_AUC += (cov_tp/tot_tp) * (fp/tot_fp) + ((tp/tot_tp) * (fp/tot_fp))/2
                cov_tp += tp

        AUCs.append(single_AUC)

    print("All AUCs:")
    print(AUCs)
    print()

    # Count how many times each col appears in correct labels
    col_correct = []
    for col in df.columns:
        correct_count = 0
        for label in correctlabels:
            if col == label:
                correct_count += 1
        print(f"{col} is correct {correct_count} times")
        col_correct.append(correct_count)

    # Compute final weighted AUC
    AUC = 0
    for i in range(len(AUCs)):
        weight = col_correct[i]/len(correctlabels)
        AUC += weight * AUCs[i]

    print()
    return AUC

In [ ]:
# Test your code  (leave this part unchanged)

predictions = pd.DataFrame({"A":[0.9,0.9,0.6,0.55],"B":[0.1, 0.1, 0.4, 0.45]})

correctlabels = ["A","B","B","A"]

# predictions = pd.DataFrame({"1":[0.6,0.4,0.3, 0.2, 0.1],"0":[0.4, 0.6, 0.7, 0.8, 0.9]})

# correctlabels = ["1","1","0","1", "0"]

print("Final weighted AUC: {}".format(auc(predictions,correctlabels)))

Triples for 1:
{0.6: [1, 0], 0.4: [1, 0], 0.3: [0, 1], 0.2: [1, 0], 0.1: [0, 1]}

Triples for 0:
{0.4: [0, 1], 0.6: [0, 1], 0.7: [1, 0], 0.8: [0, 1], 0.9: [1, 0]}

Probability ordering:
0.6
0.4
0.3
0.2
0.1

Probability ordering:
0.9
0.8
0.7
0.6
0.4

All AUCs:
[0.8333333333333333, 0.8333333333333333]

1 is correct 3 times
0 is correct 2 times

Final weighted AUC: 0.8333333333333333


In [ ]:
print("Verify results of binary classification with sklearn")
from sklearn.metrics import roc_auc_score
correct = [1,0,0,1] # A B B A
predsA = [0.9,0.9,0.6,0.55] # provide just the probabilities for the class with the greatest label

auc_skl = roc_auc_score(correct, predsA, average="weighted")
print(f"AUC with sklearn: {auc_skl}")

Verify results of binary classification with sklearn
AUC with sklearn: 0.375


In [ ]:
predictions = pd.DataFrame({"A":[0.5,0.5,0.5,0.25,0.25],"B":[0.5,0.25,0.25,0.5,0.25],"C":[0.0,0.25,0.25,0.25,0.5]})

correctlabels = ["B","A","B","B","C"]

print("Final weighted AUC: {}".format(auc(predictions,correctlabels)))

Triples for A:
{0.5: [1, 2], 0.25: [0, 2]}

Triples for B:
{0.5: [2, 0], 0.25: [1, 2]}

Triples for C:
{0.0: [0, 1], 0.25: [0, 3], 0.5: [1, 0]}

Probability ordering:
0.5
0.25

Probability ordering:
0.5
0.25

Probability ordering:
0.5
0.25
0.0

All AUCs:
[0.75, 0.8333333333333333, 1.0]

A is correct 1 times
B is correct 3 times
C is correct 1 times

Final weighted AUC: 0.8499999999999999


In [ ]:
print("Verify results of multiclass classification with sklearn")
from sklearn.metrics import roc_auc_score
correct = ["B","A","B","B","C"]

predsA = [0.5,0.5,0.5,0.25,0.25]
predsB = [0.5,0.25,0.25,0.5,0.25]
predsC = [0.0,0.25,0.25,0.25,0.5]

preds = np.array([predsA,predsB,predsC]).T
auc_skl = roc_auc_score(correct,preds,multi_class="ovr",average="weighted")
print(f"Weighted AUC with sklearn: {auc_skl}")

Verify results of multiclass classification with sklearn
Weighted AUC with sklearn: 0.85
